In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import tabula
import requests
import json
from database_utils import DatabaseConnector
from sqlalchemy import inspect

In [47]:
db_connector = DatabaseConnector()
source_credentials = db_connector.read_source_db_creds()
source_engine = db_connector.init_source_db_engine(source_credentials)
try:
    print(f"Connection to the {source_credentials['RDS_HOST']} for user {source_credentials['RDS_USER']} created successfully.")
except Exception as ex:
    print("Connection could not be made due to the following error: \n", ex)

Connection to the data-handling-project-readonly.cq2e8zno855e.eu-west-1.rds.amazonaws.com for user aicore_admin created successfully.


In [48]:
def list_db_tables(self):
        try:
            # Use the sqalchemy inspector to get table names
            inspector = inspect(source_engine)
            table_names = inspector.get_table_names()
            return table_names
        except Exception as e:
            print(f"Error listing tables: {e}")
            return None
        
list_db_tables(source_credentials)

['legacy_store_details', 'legacy_users', 'orders_table']

In [49]:
def read_rds_table(table_name):
        try:
            query = f"SELECT * FROM {table_name}"
            df = pd.read_sql_query(query, source_engine, index_col='index')
            return df
        except Exception as e:
            print(f"Error reading table {table_name}: {e}")
            return None

orders_table_df = read_rds_table("orders_table")

In order to import the data from the order's table a connection needed to be established with the database. A function was called to gather the names of the tables and then this table name could be inputted to read_rds_table code above to retrieve the data.

In [25]:
orders_table_df.head()

,level_0,date_uuid,first_name,last_name,user_uuid,card_number,store_code,product_code,1,product_quantity
index,,,,,,,,,,
0,0,9476f17e-5d6a-4117-874d-9cdb38ca1fa6,None,None,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8,30060773296197,BL-8387506C,R7-3126933h,None,3
1,1,0423a395-a04d-4e4a-bd0f-d237cbd5a295,None,None,8fe96c3a-d62d-4eb5-b313-cf12d9126a49,349624180933183,WEB-1388012W,C2-7287916l,None,2
2,2,65187294-bb16-4519-adc0-787bbe423970,None,None,fc461df4-b919-48b2-909e-55c95a03fe6b,3529023891650490,CH-01D85C8D,S7-1175877v,None,2
3,3,579e21f7-13cb-436b-83ad-33687a4eb337,None,None,6104719f-ef14-4b09-bf04-fb0c4620acb0,213142929492281,CL-C183BE4B,D8-8421505n,None,2
4,4,00ab86c3-2039-4674-b9c1-adbcbbf525bd,None,None,9523a6d3-b2dd-4670-a51a-36aebc89f579,502067329974,SO-B5B9CB3B,B6-2596063a,None,2


In [26]:
orders_table_df.describe(include="all")

,level_0,date_uuid,first_name,last_name,user_uuid,card_number,store_code,product_code,1,product_quantity
count,120123.00000,120123,15284,15284,120123,1.201230e+05,120123,120123,0,120123.000000
unique,NaN,120123,2162,1176,15284,NaN,441,1846,0,NaN
top,NaN,9476f17e-5d6a-4117-874d-9cdb38ca1fa6,Michael,Smith,745a4046-3437-4a58-a307-9c88485f45f8,NaN,WEB-1388012W,o6-5904385r,NaN,NaN
freq,NaN,1,57,340,21,NaN,26957,92,NaN,NaN
mean,60061.00000,NaN,NaN,NaN,NaN,3.902612e+17,NaN,NaN,NaN,4.010772
std,34676.66753,NaN,NaN,NaN,NaN,1.265590e+18,NaN,NaN,NaN,3.150723
min,0.00000,NaN,NaN,NaN,NaN,6.046911e+08,NaN,NaN,NaN,1.000000
25%,30030.50000,NaN,NaN,NaN,NaN,3.681678e+13,NaN,NaN,NaN,2.000000
50%,60061.00000,NaN,NaN,NaN,NaN,3.503965e+15,NaN,NaN,NaN,3.000000
75%,90091.50000,NaN,NaN,NaN,NaN,4.603236e+15,NaN,NaN,NaN,5.000000


In [28]:
orders_table_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120123 entries, 0 to 118804
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   level_0           120123 non-null  int64 
 1   date_uuid         120123 non-null  object
 2   first_name        15284 non-null   object
 3   last_name         15284 non-null   object
 4   user_uuid         120123 non-null  object
 5   card_number       120123 non-null  int64 
 6   store_code        120123 non-null  object
 7   product_code      120123 non-null  object
 8   1                 0 non-null       object
 9   product_quantity  120123 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 10.1+ MB


Interestingly, it couldn't give a unique count for card_number which means that there may be non-numeric values included.

In [29]:
orders_table_df['card_number'] = orders_table_df['card_number'].astype(object)

In [30]:
orders_table_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120123 entries, 0 to 118804
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   level_0           120123 non-null  int64 
 1   date_uuid         120123 non-null  object
 2   first_name        15284 non-null   object
 3   last_name         15284 non-null   object
 4   user_uuid         120123 non-null  object
 5   card_number       120123 non-null  object
 6   store_code        120123 non-null  object
 7   product_code      120123 non-null  object
 8   1                 0 non-null       object
 9   product_quantity  120123 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 10.1+ MB


In [32]:
orders_table_df.describe(include="all")

,level_0,date_uuid,first_name,last_name,user_uuid,card_number,store_code,product_code,1,product_quantity
count,120123.00000,120123,15284,15284,120123,1.201230e+05,120123,120123,0,120123.000000
unique,NaN,120123,2162,1176,15284,1.528400e+04,441,1846,0,NaN
top,NaN,9476f17e-5d6a-4117-874d-9cdb38ca1fa6,Michael,Smith,745a4046-3437-4a58-a307-9c88485f45f8,3.027473e+13,WEB-1388012W,o6-5904385r,NaN,NaN
freq,NaN,1,57,340,21,2.100000e+01,26957,92,NaN,NaN
mean,60061.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.010772
std,34676.66753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.150723
min,0.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
25%,30030.50000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000
50%,60061.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000
75%,90091.50000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000


In [33]:
orders_table_df['card_number'].nunique()

15284

Perfect - this has the same count as the card_details table.

In [34]:
# Create a boolean mask for non-numeric values in the specified column
non_numeric_mask = ~pd.to_numeric(orders_table_df['card_number'], errors='coerce').notna()

# Use the mask to display the rows where the specified column has non-numeric values
rows_with_non_numeric_values = orders_table_df[non_numeric_mask]

# Display the rows where the specified column has non-numeric values
print(rows_with_non_numeric_values)

Empty DataFrame
Columns: [level_0, date_uuid, first_name, last_name, user_uuid, card_number, store_code, product_code, 1, product_quantity]
Index: []


In [35]:
orders_table_df.isnull().sum()

level_0                  0
date_uuid                0
first_name          104839
last_name           104839
user_uuid                0
card_number              0
store_code               0
product_code             0
1                   120123
product_quantity         0
dtype: int64

In [36]:
columns_to_drop = ['first_name', 'last_name', '1']
orders_table_df = orders_table_df.drop(columns=columns_to_drop, errors='ignore')

In [37]:
orders_table_df.head()

,level_0,date_uuid,user_uuid,card_number,store_code,product_code,product_quantity
index,,,,,,,
0,0,9476f17e-5d6a-4117-874d-9cdb38ca1fa6,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8,30060773296197,BL-8387506C,R7-3126933h,3
1,1,0423a395-a04d-4e4a-bd0f-d237cbd5a295,8fe96c3a-d62d-4eb5-b313-cf12d9126a49,349624180933183,WEB-1388012W,C2-7287916l,2
2,2,65187294-bb16-4519-adc0-787bbe423970,fc461df4-b919-48b2-909e-55c95a03fe6b,3529023891650490,CH-01D85C8D,S7-1175877v,2
3,3,579e21f7-13cb-436b-83ad-33687a4eb337,6104719f-ef14-4b09-bf04-fb0c4620acb0,213142929492281,CL-C183BE4B,D8-8421505n,2
4,4,00ab86c3-2039-4674-b9c1-adbcbbf525bd,9523a6d3-b2dd-4670-a51a-36aebc89f579,502067329974,SO-B5B9CB3B,B6-2596063a,2


In [38]:
orders_table_df.describe(include="all")

,level_0,date_uuid,user_uuid,card_number,store_code,product_code,product_quantity
count,120123.00000,120123,120123,1.201230e+05,120123,120123,120123.000000
unique,NaN,120123,15284,1.528400e+04,441,1846,NaN
top,NaN,9476f17e-5d6a-4117-874d-9cdb38ca1fa6,745a4046-3437-4a58-a307-9c88485f45f8,3.027473e+13,WEB-1388012W,o6-5904385r,NaN
freq,NaN,1,21,2.100000e+01,26957,92,NaN
mean,60061.00000,NaN,NaN,NaN,NaN,NaN,4.010772
std,34676.66753,NaN,NaN,NaN,NaN,NaN,3.150723
min,0.00000,NaN,NaN,NaN,NaN,NaN,1.000000
25%,30030.50000,NaN,NaN,NaN,NaN,NaN,2.000000
50%,60061.00000,NaN,NaN,NaN,NaN,NaN,3.000000
75%,90091.50000,NaN,NaN,NaN,NaN,NaN,5.000000


In [42]:
search_value = "2720312980409662"
orders_table_df[orders_table_df['card_number'].astype(str).str.contains(search_value)]

,level_0,date_uuid,user_uuid,card_number,store_code,product_code,product_quantity
index,,,,,,,
4288,4288,960a3153-05b1-4888-bd63-d6757b47e8d9,b1a4aaa2-def5-4baa-a3c9-1c7d42ac92cd,2720312980409662,WEB-1388012W,Y0-0466444e,4
29878,29878,5d8b5bdf-f82d-4f82-9fc7-443d5e419c3e,1c812069-8f7d-4fd8-937a-7805e39eaf88,2720312980409662,TH-B6A0AB51,T8-6501161g,6
63903,63904,652acf9c-3e30-4e60-8ee5-5a5c9d1292c0,8953c7e3-2591-41a1-9511-c23020b7e631,2720312980409662,WEB-1388012W,h8-0983805L,15
90057,90057,8783681a-dfca-4ce8-b6b3-86f7313978cf,e8c22782-8f38-4c73-a2bd-763700ec9cd0,2720312980409662,HA-FC4E6AFF,R2-6636631H,4
90176,90176,1e0ae6a1-1d91-41f8-9353-b2d9049fe59b,5dccfa21-eae1-4698-a01c-baecf9e3a860,2720312980409662,WEB-1388012W,j1-4776534u,3
96832,96832,e277393c-c22d-4cfa-8e8d-d28f00c3269c,c2672ddc-0734-4bd3-aa1c-fa5fa607c065,2720312980409662,EX-2C210EBF,a8-4156593e,3


This was to check that the card_numbers with removed "?" now match the card_numbers in the orders_table. They do.